In [ ]:
# default_exp core

# pytrim2

> A python program for trimming and demultiplexing naopore reads.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
# Import dependencies
from Bio import SeqIO
from Bio import pairwise2
import numpy as np

In [ ]:
#export

def findAdapter(seq_record, barcode_primer):
    # Alignment using pairwise2
    alignments = pairwise2.align.localms(seq_record.seq, barcode_primer, 1, 0, -2, -2)

    # Find best match
    alingmentAccuracy = np.array( [i.score for i in alignments]  )
    alingmentEnd = np.array( [i.end for i in alignments]  )
    maxAcc = np.max(alingmentAccuracy)
    maxAcc = np.max(alingmentAccuracy)
    maxAccAlignment = np.where(alingmentAccuracy == maxAcc)[0]
    inwardAlingmentEnd = np.max(alingmentEnd[maxAccAlignment])
    inwardAlingmentEnd

    if maxAcc > 90:
        print("TRUE")

In [ ]:
#hide

# Import test data and run findAdapter on test data
